In [1]:
# Importation
import numpy as np
import pandas as pd
import plotly.express as px
from  rouge_score  import  rouge_scorer
import re
from difflib import SequenceMatcher

# **I - Data management**

In [11]:
# Chargement des données
df = pd.read_csv("../data/01_251021_Notations200.csv")

## Présentation des données 


Le jeu de données qu'on traite dans le notebook est issue du jeu de données open source  [La fiscalité et les dépenses publiques](https://www.data.gouv.fr/fr/datasets/r/bc085888-e6bd-445d-b3f4-632190c29e3f). Celui-ci est notament constitué des réponses textuelles des contribuables à des questions portant sur la fiscalité en France. Dans cette étude, on se penche principalement sur la question suivante  : *Que faudrait-il faire pour rendre la fiscalité plus juste et plus efficace ?*.

Pour extraire idées majeures des réponses textuelles, on utilise le modèle de langue ollama : *hugging-quants/Meta-Llama-3.1-8B-Instruct-AWQ-INT4*. Avec les hyperparamètres **top_p = 0.95**
et **temperature = 0**.

On donne à ce LLM le prompt suivant :


[prompt]
system = """
But: extraire les idées principales DISTINCTES d’un texte pour analyse.

Règles:
1. N’utiliser QUE le contenu entre <<< TEXT >>>.
2. Extraire la liste des idées DISTINCTES et PRINCIPALES.
   - Chaque idée = une phrase claire, autonome, reformulée si nécessaire.
3. Pour CHAQUE idée, annoter:
   - type: "statement" (constat) OU "proposition" (suggestion/recommandation/objectif).
   - syntax: "negative" si la phrase contient une négation explicite (ex.: "ne", "n'", "ne pas", "ne plus", "non"), sinon "positive".
   - semantic: "positive", "negative" ou "neutral" (valence sémantique).
4. Sortie STRICTEMENT en CSV avec entête EXACTE:
   CSV:description,type,syntax,semantic
   - Délimiteur: virgule.
   - Chaque description entre guillemets doubles.
   - Échapper tout guillemet interne par duplication (ex.: ""chat"").
   - NE RIEN AJOUTER d’autre (pas de texte avant/après, pas de code fences).
   - Pas de lignes vides.

Exemple:
CSV:description,type,syntax,semantic
"Les chats retombent sur leurs pattes",statement,positive,neutral
"Les chats n'ont pas neuf vies",statement,negative,negative
"Il faut mieux prendre soin des chats pour prolonger leur vie",proposition,positive,positive
"""
user = """<<< {input} >>>"""

Ainsi, on construit le jeu de données **df**. Ces variables sont les suivantes :

- **authorId** : Identifiant de l’auteur
- **contrib_index** : Numéro de la contribution
- **contribution** : Texte original de l'auteur
- **C** : Score calculé par la métrique *QualIT*
- **n_ideas** : Nombre d’idées extraites
- **len_contrib** : Longueur du texte original
- **Matthias, Yannis, Garance** : Note allant de 0 à 10 évaluant selon la personne la qualité de l'extraction
- **Hallucinations** : Présence d'hallucination dans l'extraction
- **Idées_non_ind** : Idées dénuée de sens dans l'extraction

On construit ensuite de nouvelles variables à partir de ces données brutes pour faciliter l'analyse :

In [12]:
# Calcul du score humain moyen
df["score_humain"] = df[["Garance", "Matthias", "Yannis"]].mean(axis=1)

# Copie des colonnes d'hallucinations/d'idées invalides en version catégorielle
mapping = {0:"Non", 1:"Oui"}
df["pres_hallu"] = df["Hallucinations"].map(mapping)
df["pres_idees_inv"] = df["Idées_non_ind"].map(mapping)

# Nombre de tokens par contribution et par extractions (comptage simple par espaces)
df["nb_tokens_contrib"] = df["contribution"].apply(lambda x: len(str(x).split()))
df["nb_tokens_extraction"] = df["ideas_text"].apply(lambda x: len(str(x).split()))

# **II - Validité de la notation et de la métrique**

In [13]:
# Calcul des corrélations de Pearson entre chaque paire d'évaluateurs
corr_GM = df["Garance"].corr(df["Matthias"], method='pearson')
corr_GY = df["Garance"].corr(df["Yannis"], method='pearson')
corr_MY = df["Matthias"].corr(df["Yannis"], method='pearson')
# Corrélation de Pearson entre score humain et score QualIT
correlation = df["score_humain"].corr(df["C"], method='pearson')

print(f"Corrélation (de Pearson) entre Garance et Matthias : {corr_GM:.2f}")
print(f"Corrélation (de Pearson) entre Garance et Yannis : {corr_GY:.2f}")
print(f"Corrélation (de Pearson) entre Matthias et Yannis : {corr_MY:.2f}")
print(f"Corrélation entre le score humain moyen et le score QualIT : {correlation:.2f}")

Corrélation (de Pearson) entre Garance et Matthias : 0.93
Corrélation (de Pearson) entre Garance et Yannis : 0.93
Corrélation (de Pearson) entre Matthias et Yannis : 0.94
Corrélation entre le score humain moyen et le score QualIT : 0.71


Les trois évaluateurs humains (Matthias, Yannis, Garance) semblent globalement en accord.

La métrique QualIT et les notes humaines sont assez fortement corrélées, bien que moins que les notes entre évaluateurs humains. La métrique semble donc assez fiable pour évaluer la qualité des extractions. Nous pouvons toutefois observer quelques visualisations afin d'identifier cette baisse de corrélation.

# **III - Visualisations**

### **A - Caractéristiques des contributions**

In [10]:
# Barplot du nombre de charactères par contribution sur les 200 premières contributions
fig = px.histogram(
    df, x="nb_tokens_contrib",
    color_discrete_sequence=["#2a2781"]
)
fig.update_layout(
    title={"text": "<b>Distribution du nombre de tokens par contribution</b>"}, 
    xaxis_title="Nombre de tokens", 
    yaxis_title="Nombre de contributions", 
    width=800, height=450
)
fig.update_traces(
    hovertemplate =
        "Nombre de tokens = %{x}<br>" \
        "Nombre de contributions = %{y}<extra></extra>"
)
fig.show()

Au vu de la distribution, il semble judicieux de regrouper les contributions en différentes catégories de longueur (arbitraire).

In [11]:
# Répartition des contributions en 5 catégories équilibrées selon le nombre de tokens (basé sur les quantiles)
df["contrib_tokens_bins"] = pd.qcut(df["nb_tokens_contrib"], q=5, labels=[
    "Très court", "Court", "Moyen", "Long", "Très long"
])

# Autre options : catégories basées sur des seuils fixes
df["contrib_tokens_bins_fixe"] = pd.cut(df["nb_tokens_contrib"], bins=[0, 50, 100, 200, np.inf], labels=[
    "Très court (0-50)", "Court (51-100)", "Moyen (101-200)", "Long (+200)"
])

### **B - Caractéristiques des extractions**

In [12]:
# Version avec les bins équilibrés
fig1 = px.histogram(
    df, x="contrib_tokens_bins", y="Hallucinations",
    color_discrete_sequence=["#2a2781"], histfunc="avg", category_orders={
        "contrib_tokens_bins": ["Très court", "Court", "Moyen", "Long", "Très long"]
    }
)
fig1.update_layout(
    title={"text": 
        "<b>Distribution du taux d'hallucinations par catégories <br>équilibrées de longueur</b>"
    }, 
    xaxis_title="Catégories de longueur (en nombre de tokens)", yaxis_title="Taux d'hallucinations",
    width=700, height=450
)
fig1.update_traces(
    hovertemplate =
        "Taux d'hallucinations = %{y}<extra></extra>"
)
# Version avec les bins fixes
fig2 = px.histogram(
    df, x="contrib_tokens_bins_fixe", y="Hallucinations",
    color_discrete_sequence=["#2a2781"], histfunc="avg", category_orders={
        "contrib_tokens_bins_fixe": ["Très court (0-50)", "Court (51-100)", "Moyen (101-200)", "Long (+200)"]
    }
)
fig2.update_layout(
    title={"text": 
        "<b>Distribution du taux d'hallucinations par catégories <br>arbitraire de longueur</b>"
    }, 
    xaxis_title="Catégories de longueur (en nombre de tokens)", yaxis_title="Taux d'hallucinations",
    width=700, height=450
)
fig2.update_traces(
    hovertemplate =
        "Taux d'hallucinations = %{y}<extra></extra>"
)

# Affichage des figures
fig1.show()
fig2.show()

In [13]:
# Version avec les bins équilibrés
fig1 = px.histogram(
    df, x="contrib_tokens_bins", y="Idées_non_ind",
    color_discrete_sequence=["#2a2781"], histfunc="avg", category_orders={
        "contrib_tokens_bins": ["Très court", "Court", "Moyen", "Long", "Très long"]
    }
)
fig1.update_layout(
    title={"text": 
        "<b>Distribution du taux d'idées invalides par catégories <br>équilibrées de longueur</b>"
    }, 
    xaxis_title="Catégories de longueur (en nombre de tokens)", yaxis_title="Taux d'idées invalides",
    width=700, height=450
)
fig1.update_traces(
    hovertemplate =
        "Taux d'idées invalides = %{y}<extra></extra>"
)
# Version avec les bins fixes
fig2 = px.histogram(
    df, x="contrib_tokens_bins_fixe", y="Idées_non_ind",
    color_discrete_sequence=["#2a2781"], histfunc="avg", category_orders={
        "contrib_tokens_bins_fixe": ["Très court [0;50]", "Court [51;100]", "Moyen [101;200]", "Long +200"]
    }
)
fig2.update_layout(
    title={"text": 
        "<b>Distribution du taux d'idées invalides par catégories <br>arbitraire de longueur</b>"
    }, 
    xaxis_title="Catégories de longueur (en nombre de tokens)", yaxis_title="Taux d'idées invalides",
    width=700, height=450
)
fig2.update_traces(
    hovertemplate =
        "Taux d'idées invalides = %{y}<extra></extra>"
)

# Affichage des figures
fig1.show()
fig2.show()

### **C - Caractéristiques des évaluations**

Attention : Les extractions échouées ne sont pas sur la plupart des graphiques car elles n'ont pas de valeur dans "Hallucinations" ou "Idées_non_ind".

In [14]:
# Heatmap de densité en arrière-plan
fig = px.density_contour(
    df, x="C", y="score_humain",
    nbinsx=20, nbinsy=20
)
fig.update_traces(contours_coloring="fill", contours_showlabels = False, colorscale="Blues")
# Scatter plot
fig.add_scatter(
    x=df["C"], y=df["score_humain"],
    mode="markers",
    marker=dict(color="#000000"),
    hovertemplate="Métrique QualIT = %{x}<br>Score humain = %{y}<extra></extra>"
)
fig.add_shape(
    type="line", x0=0, x1=1, y0=0, y1=10,
    xref="x", yref="y",
    line=dict(color="#d41010")
)
fig.update_layout(
    title="<b>Score humain vs métrique QualIT</b>",
    xaxis_title="Métrique QualIT", yaxis_title="Score humain",
    width=700, height=500,
    coloraxis_showscale=False
)
fig.show()

In [15]:
# Coloration avec hallucinations - version linéaire
fig = px.scatter(
    df, x="C", y="score_humain", 
    color="pres_hallu", color_discrete_sequence=["#ff6361", "#003f5c"]
)
fig.add_shape(       # Ajout de la ligne x = y pour référence
    type="line", x0=0, x1=1, y0=0, y1=10, xref="x", yref="y",
    line=dict(color="#d41010")
)
fig.update_layout(
    title={"text": "<b>Score humain vs métrique QualIT</b>"},
    xaxis_title="Métrique QualIT", yaxis_title="Score humain", 
    legend_title="Présence <br>d'hallucinations",
    width=700, height=500
)
fig.update_traces(
    hovertemplate =
        "Métrique QualIT = %{x}<br>" \
        "Score humain = %{y}<extra></extra>"
)
fig.show()

In [16]:
# Coloration avec hallucinations - version logarithmique
df["log_score_humain"] = np.log10(df["score_humain"]+1)

fig = px.scatter(
    df, x="C", y="log_score_humain", 
    color="pres_hallu", color_discrete_sequence=["#ff6361", "#003f5c"]
)
fig.add_shape(      # Ajout de la ligne x = y pour référence
    type="line", x0=0, x1=1, y0=0, y1=1, xref="x", yref="y",
    line=dict(color="#d41010")
)
fig.update_layout(
    title={"text": "<b>log(Score humain + 1) vs métrique QualIT</b>"},
    xaxis_title="Métrique QualIT", yaxis_title="log(Score humain + 1)", 
    legend_title="Présence <br>d'hallucinations",
    width=700, height=500
)
fig.update_traces(
    hovertemplate =
        "Métrique QualIT = %{x}<br>" \
        "log(Score humain + 1) = %{y}<extra></extra>"
)
fig.show()

In [17]:
# Coloration avec hallucinations - version linéaire
fig = px.scatter(
    df, x="C", y="score_humain", 
    color="pres_idees_inv", color_discrete_sequence=["#ff6361", "#003f5c"]
)
fig.add_shape(       # Ajout de la ligne x = y pour référence
    type="line", x0=0, x1=1, y0=0, y1=10, xref="x", yref="y",
    line=dict(color="#d41010")
)
fig.update_layout(
    title={"text": "<b>Score humain vs métrique QualIT</b>"},
    xaxis_title="Métrique QualIT", yaxis_title="Score humain", 
    legend_title="Présence d'idées <br>invalides",
    width=700, height=500
)
fig.update_traces(
    hovertemplate =
        "Métrique QualIT = %{x}<br>" \
        "Score humain = %{y}<extra></extra>"
)
fig.show()

In [18]:
# Version avec les bins équilibrées
fig1 = px.scatter(
    df, x="C", y="score_humain", color="contrib_tokens_bins", 
    color_discrete_sequence=["#ffd380", "#ff6361", "#8a508f", "#003348", "#000000"],
    category_orders={
        "contrib_tokens_bins": ["Très court", "Court", "Moyen", "Long", "Très long"]
    }, 
    symbol="pres_idees_inv", symbol_map={"Oui":"x", "Non":"circle"},
)
fig1.add_shape(       # Ajout de la ligne x = y pour référence
    type="line", x0=0, x1=1, y0=0, y1=10, xref="x", yref="y",
    line=dict(color="#d41010")
)
fig1.update_layout(
    title={"text": "<b>Score humain vs la métrique QualIT</b>"},
    xaxis_title="Métrique QualIT", yaxis_title="Score humain", 
    legend_title="Longueur, présence <br>d'idées invalides",
    width=700, height=500
)
fig1.update_traces(
    hovertemplate =
        "Métrique QualIT = %{x}<br>" \
        "Score humain = %{y}<extra></extra>"
)
# Version avec les bins fixes
fig2 = px.scatter(
    df, x="C", y="score_humain", color="contrib_tokens_bins_fixe", 
    color_discrete_sequence=["#ffd380", "#ff6361", "#8a508f", "#000000"],
    category_orders={
        "contrib_tokens_bins_fixe": ["Très court (0-50)", "Court (51-100)", "Moyen (101-200)", "Long (+200)"]
    },
    symbol="pres_idees_inv", symbol_map={"Oui":"circle", "Non":"x"},
)
fig2.add_shape(       # Ajout de la ligne x = y pour référence
    type="line", x0=0, x1=1, y0=0, y1=10, xref="x", yref="y",
    line=dict(color="#d41010")
)
fig2.update_layout(
    title={"text": "<b>Score humain vs la métrique QualIT</b>"},
    xaxis_title="Métrique QualIT", yaxis_title="Score humain", 
    legend_title="Longueur, présence <br>d'idées invalides",
    width=700, height=500
)
fig2.update_traces(
    hovertemplate =
        "Métrique QualIT = %{x}<br>" \
        "Score humain = %{y}<extra></extra>"
)

# Affichage des figures
fig1.show()
fig2.show()

In [19]:
# Version avec les bins équilibrées
fig1 = px.scatter(
    df, x="C", y="score_humain", color="contrib_tokens_bins", 
    color_discrete_sequence=["#ffd380", "#ff6361", "#8a508f", "#003348", "#000000"],
    category_orders={
        "contrib_tokens_bins": ["Très court", "Court", "Moyen", "Long", "Très long"]
    }, 
    symbol="pres_hallu", symbol_map={"Oui":"x", "Non":"circle"},
)
fig1.add_shape(       # Ajout de la ligne x = y pour référence
    type="line", x0=0, x1=1, y0=0, y1=10, xref="x", yref="y",
    line=dict(color="#d41010")
)
fig1.update_layout(
    title={"text": "<b>Score humain vs la métrique QualIT</b>"},
    xaxis_title="Métrique QualIT", yaxis_title="Score humain", 
    legend_title="Longueur, présence <br>d'hallucinations",
    width=700, height=500
)
fig1.update_traces(
    hovertemplate =
        "Métrique QualIT = %{x}<br>" \
        "Score humain = %{y}<extra></extra>"
)
# Version avec les bins fixes
fig2 = px.scatter(
    df, x="C", y="score_humain", color="contrib_tokens_bins_fixe", 
    color_discrete_sequence=["#ffd380", "#ff6361", "#8a508f", "#000000"],
    category_orders={
        "contrib_tokens_bins_fixe": ["Très court (0-50)", "Court (51-100)", "Moyen (101-200)", "Long (+200)"]
    },
    symbol="pres_hallu", symbol_map={"Oui":"circle", "Non":"x"},
)
fig2.add_shape(       # Ajout de la ligne x = y pour référence
    type="line", x0=0, x1=1, y0=0, y1=10, xref="x", yref="y",
    line=dict(color="#d41010")
)
fig2.update_layout(
    title={"text": "<b>Score humain vs la métrique QualIT</b>"},
    xaxis_title="Métrique QualIT", yaxis_title="Score humain", 
    legend_title="Longueur, présence <br>d'hallucinations",
    width=700, height=500
)
fig2.update_traces(
    hovertemplate =
        "Métrique QualIT = %{x}<br>" \
        "Score humain = %{y}<extra></extra>"
)

# Affichage des figures
fig1.show()
fig2.show()

In [20]:
# Version avec les bins équlibrées
fig1 = px.histogram(
    df, x="contrib_tokens_bins", y="C", histfunc="avg", color="contrib_tokens_bins",
    color_discrete_sequence=["#ffd380", "#ff6361", "#8a508f", "#003348", "#000000"],
    category_orders={
        "contrib_tokens_bins": ["Très court", "Court", "Moyen", "Long", "Très long"]
    }
)
fig1.update_layout(
    title={"text": "<b>Métrique QualIT par catégorie de longueur<b>"}, 
    xaxis_title="Catégories de longueur (en nombre de tokens)", 
    yaxis_title="Moyenne QualIT", 
    width=800, height=450, showlegend=False
)
fig1.update_traces(
    hovertemplate =
        "Catégorie de longueur = %{x}<br>" \
        "Moyenne QualIT = %{y}<extra></extra>"
)
# Version avec les bins fixes
fig2 = px.histogram(
    df, x="contrib_tokens_bins_fixe", y="C", histfunc="avg", color="contrib_tokens_bins_fixe",
    color_discrete_sequence=["#ffd380", "#ff6361", "#8a508f", "#000000"],
    category_orders={
        "contrib_tokens_bins_fixe": ["Très court (0-50)", "Court (51-100)", "Moyen (101-200)", "Long (+200)"]
    }
)
fig2.update_layout(
    title={"text": "<b>Métrique QualIT par catégorie de longueur<b>"}, 
    xaxis_title="Catégories de longueur (en nombre de tokens)", 
    yaxis_title="Moyenne QualIT", 
    width=800, height=450, showlegend=False
)
fig2.update_traces(
    hovertemplate =
        "Catégorie de longueur = %{x}<br>" \
        "Moyenne QualIT = %{y}<extra></extra>"
)

# Affichage des figures
fig1.show()
fig2.show()

## Métrique ROUGE

les métrique ROUGE (Recall-Oriented Understudy for Gisting Evaluation ) est un ensemble de métriques qui sont généralement utilisées pour évaluier la qualité des résumés proposées par un LLM. Ces différentes métriques repose dans notre cas sur les similarités entre les mots de l'extraction du LLM et l'idée de base. On peut définir 3 catégories de métriques ROUGE :
- N-grammes : compare les groupes de N mots consécutifs entre l'idée originel et l'extraction
- L ( Longest Common Subsequence) : mesure la similarité en séquence plutot qu'en N-gramm
- S/SU : Statistique de co-occurence reposant sur les bigrammes de saut (paire de mots quelconque dans leur ordre de phrase): bigramme sauté et les unigrammes

Celles-ci sont comprises entre 0 et 1 : plus la métrique est proche de 1 et plus l'extraction est de bonne qualité. Inversement, plus la métrique est proche de 0 et plus elle est de mauvaise qualité.

Dans notre cas, on cherche à évaluer la qualité des idées extraites plus qu'une similarité mot à mot. La métrique la plmus adapté semble donc etre la métrique LCS. 
Pour implémenter cette métrique avec Python, on emploiera le package [rouge-score](https://pypi.org/project/rouge-score/). L'exemple de base sur ce site calcule la métrique rouge &-gramm et L, ce que l'on va calculer puis implémenter dans le jeud e donnée df 

In [20]:
# On crée les vars pour la métrique ROUGE
df["1gramm"],df["L"] = 0,0

In [21]:
# Les 2 métriques à évaluer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

for i in range(np.shape(df)[0]):
    # try pour que ca passe avec les parse error
    try:
        ref = str(df.loc[i, 'ideas_text'])
        pred = str(df.loc[i, 'contribution'])
        
        scores = scorer.score(ref, pred)
        
        df.loc[i, '1gramm'] = scores['rouge1'].fmeasure
        df.loc[i, 'L'] = scores['rougeL'].fmeasure
    
    # 0 si ca ne passe pas
    except Exception as e:
        print(f"Erreur à la ligne {i} : {e}")
        df.loc[i, '1gramm'] = 0
        df.loc[i, 'L'] = 0

/var/folders/bm/q285w4797r77bpfkn5xxc05w0000gn/T/ipykernel_16128/2714954936.py:12: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.0014492753623188406' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/var/folders/bm/q285w4797r77bpfkn5xxc05w0000gn/T/ipykernel_16128/2714954936.py:13: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.0014492753623188406' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.



In [22]:
# Corrélation de Pearson entre score humain et score ROUGE &gramm/L
correlation_1gramm = df["score_humain"].corr(df["1gramm"])
print(f"Corrélation entre le score humain moyen et le score ROUGE 1gramm : {correlation_1gramm:.2f}")

correlation_L = df["score_humain"].corr(df["L"])
print(f"Corrélation entre le score humain moyen et le score ROUGE L : {correlation_L:.2f}")

Corrélation entre le score humain moyen et le score ROUGE 1gramm : 0.79
Corrélation entre le score humain moyen et le score ROUGE L : 0.78


Les corrélation entre l'évaluation humaine et celles des métriques ROUGE est plus grande que celle de QualIT (0.71)

## Plot des métriques rouges

In [23]:
# Heatmap de densité en arrière-plan
fig = px.density_contour(
    df, x="1gramm", y="score_humain",
    nbinsx=20, nbinsy=20
)
fig.update_traces(contours_coloring="fill", contours_showlabels = False, colorscale="Blues")
# Scatter plot
fig.add_scatter(
    x=df["1gramm"], y=df["score_humain"],
    mode="markers",
    marker=dict(color="#000000"),
    hovertemplate="Métrique ROUGE 1-gramm = %{x}<br>Score humain = %{y}<extra></extra>"
)
fig.add_shape(
    type="line", x0=0, x1=1, y0=0, y1=10,
    xref="x", yref="y",
    line=dict(color="#d41010")
)
fig.update_layout(
    # title="<b>Score humain vs Métrique ROUGE 1-gramm</b>",
    xaxis_title="1-gram ROUGE metric", yaxis_title="Human score",
    width=700, height=500,
    coloraxis_showscale=False
)
fig.show()

In [24]:
# Heatmap de densité en arrière-plan
fig = px.density_contour(
    df, x="L", y="score_humain",
    nbinsx=20, nbinsy=20
)
fig.update_traces(contours_coloring="fill", contours_showlabels = False, colorscale="Blues")
# Scatter plot
fig.add_scatter(
    x=df["L"], y=df["score_humain"],
    mode="markers",
    marker=dict(color="#000000"),
    hovertemplate="Métrique ROUGE L = %{x}<br>Score humain = %{y}<extra></extra>"
)
fig.add_shape(
    type="line", x0=0, x1=1, y0=0, y1=10,
    xref="x", yref="y",
    line=dict(color="#d41010")
)
fig.update_layout(
    # title="<b>Score humain vs Métrique ROUGE L</b>",
    xaxis_title="L ROUGE metric", yaxis_title="Human score",
    width=700, height=500,
    coloraxis_showscale=False
)
fig.show()

# Métrique NLI


La métrique NLI ( Natural Language Inference ) est une métrique qui vise à mesurer si une hypothèse $h$ est induite par une prémisse $p$, qu'elle soit contradictoire ou bien neutre.

Pour ce faire, on va définir :
- les prémises $p$ comme nos contributions
- Les hypothèses $h$ comme les idées extraites par le LLM

L'idée de la métrique est la suivante :
Pour chaques idées $h_i$, nous calculons le meilleur score de similarité avec la phrase de contribution. Nous utilisons un mélange de deux méthodes pour cela :
-*Jaccard tokenks* : C'est un recouvrement du vocbulaire
-*LCS ratio* : C'est une métrique ROUGE-L simplifiée
- On calcule ensuite la combinaison suivante :

$$ support(h_i) = max_j(0.6Jaccard(h_i,p_j)+0.4LCS(h_i,p_j))$$


Ensuite, on détecte la présence de négation dans $h_i$ et $p_j$>
Si la meilleure phrase $p_j$ ( celle qui maximise le support de $support(h_i)$) mais dispose d'une négation différente de $h_i$, on ajoute une pénalité sur la métrique NLI en fonction d'un facteur $\alpha \in ]0,1[$ :

$$
\mathrm{contra}(h_i) =
\mathbf{1}\!\left[\mathrm{neg}(h_i)\neq \mathrm{neg}\!\left(p_{j^\ast}\right)\right]
\cdot
\left(1-\mathrm{support}(h_i)\right)
\cdot
\alpha
$$




Pour obtenir le score final de la métrique, nous moyennenons sur toutes les idées :
- *NLI_support* : varie de 0 à 1. Plus nous sommes proches de ! et plus la métrique est entrainée (bonne)
- *NLI_contra* : varie de 0 à 1. Plus nous somme proches de 1 et plus les idées entre extractions et contributions sont opposées
- On calcule *NLI_final* comme la différence de NLI_support et NLI_contra

In [ ]:
# Jaccard et LCS ratio
_SENT_SPLIT = re.compile(r'(?<=[\.\?\!])\s+|\n+')
_WORD = re.compile(r"[A-Za-zÀ-ÖØ-öø-ÿ0-9']+")

NEG_MARKERS = {
    "ne", "n", "pas", "plus", "jamais", "aucun", "aucune", "sans", "ni", "rien", "personne"
}

STOPWORDS_FR_MINI = {
    # mini stoplist (évite de dépendre d'un package)
    "le","la","les","un","une","des","du","de","d","et","ou","à","a","au","aux",
    "en","dans","sur","pour","par","avec","sans","ce","cet","cette","ces",
    "que","qui","quoi","dont","où","est","sont","être","été","être","il","elle",
    "ils","elles","on","nous","vous","je","tu","se","sa","son","ses","leur","leurs",
    "mais","donc","car","si","comme","plus","moins","très"
}

def split_sentences(text: str):
    if not isinstance(text, str) or not text.strip():
        return []
    text = re.sub(r"\s+", " ", text.strip())
    return [s.strip() for s in _SENT_SPLIT.split(text) if s and s.strip()]

def tokenize(text: str):
    if not isinstance(text, str):
        return []
    toks = [t.lower() for t in _WORD.findall(text)]
    return toks

def content_tokens(text: str):
    toks = tokenize(text)
    return [t for t in toks if t not in STOPWORDS_FR_MINI and len(t) > 2]

def has_negation(text: str):
    toks = tokenize(text)
    return any(t in NEG_MARKERS for t in toks) or "n'" in text.lower()

def jaccard(a_tokens, b_tokens):
    A, B = set(a_tokens), set(b_tokens)
    if not A or not B:
        return 0.0
    return len(A & B) / len(A | B)

def lcs_ratio(a: str, b: str):
    if not a or not b:
        return 0.0
    return SequenceMatcher(None, a.lower(), b.lower()).ratio()

def parse_ideas_text(ideas_text: str):
    if not isinstance(ideas_text, str) or not ideas_text.strip():
        return []
    ideas = split_sentences(ideas_text)
    ideas = [x.strip() for x in ideas if len(x.strip()) > 3]
    return ideas



In [ ]:
# Score NLI

def nli_lexical_scores(premise: str, ideas_text: str, alpha_contra: float = 0.8):
    premise_sents = split_sentences(premise)
    if not premise_sents:
        return np.nan, np.nan, np.nan

    ideas = parse_ideas_text(ideas_text)
    if not ideas:
        return np.nan, np.nan, np.nan

    contra_scores = []
    support_scores = []

    for h in ideas:
        h_tok = content_tokens(h)
        h_neg = has_negation(h)

        best_support = 0.0
        best_sent = None

        for ps in premise_sents:
            ps_tok = content_tokens(ps)
            s_j = jaccard(h_tok, ps_tok)
            s_l = lcs_ratio(h, ps)
            support = 0.6 * s_j + 0.4 * s_l
            if support > best_support:
                best_support = support
                best_sent = ps

        support_scores.append(best_support)

        if best_sent is None:
            contra_scores.append(0.0)
        else:
            ps_neg = has_negation(best_sent)
            mismatch = 1.0 if (h_neg != ps_neg) else 0.0
            contra = mismatch * (1.0 - best_support) * alpha_contra
            contra_scores.append(contra)

    support_mean = float(np.mean(support_scores)) if support_scores else np.nan
    contra_mean = float(np.mean(contra_scores)) if contra_scores else np.nan
    final = float(np.clip(support_mean - contra_mean, 0.0, 1.0)) if np.isfinite(support_mean) and np.isfinite(contra_mean) else np.nan

    return support_mean, contra_mean, final


In [16]:
# Application à nos données
df["NLI_support"] = np.nan
df["NLI_contra"] = np.nan
df["NLI_final"] = np.nan

for i in range(len(df)):
    premise = str(df.loc[i, "contribution"]) if "contribution" in df.columns else ""
    ideas_text = str(df.loc[i, "ideas_text"]) if "ideas_text" in df.columns else ""
    s, c, f = nli_lexical_scores(premise, ideas_text, alpha_contra=0.8)
    df.loc[i, "NLI_support"] = s
    df.loc[i, "NLI_contra"] = c
    df.loc[i, "NLI_final"] = f

df[["NLI_support", "NLI_contra", "NLI_final"]].describe()

,NLI_support,NLI_contra,NLI_final
count,200.000000,200.000000,200.000000
mean,0.420649,0.162663,0.350180
std,0.251018,0.263954,0.294073
min,0.000000,0.000000,0.000000
25%,0.221919,0.000000,0.060260
50%,0.401881,0.000000,0.322630
75%,0.567480,0.326125,0.537564
max,1.000000,0.773922,1.000000


### Remarque méthodologique : contrainte d’indépendance et implémentation à la main

L’article SelfCheckGPT propose une variante SelfCheckGPT-NLI basée sur un classifieur NLI  afin d’estimer une probabilité de contradiction entre une phrase et un contexte. Dans l’implémentation, cela nécessite l’utilisation de bibliothèques de deep learning ainsi que le téléchargement et l’exécution d’un modèle pré-entraîné.

Dans ce projet, l’objectif est de conserver une métrique indépandante du LLM.

De plus, notre environnement d’exécution ne permet pas d’installer correctement `torch`/`transformers` (python 3.13 pour ma part). Pour ces raisons, nous n’implémentons pas la version “modèle NLI” de SelfCheckGPT, mais une approximation NLI lexicale calculée à la main comme pour la métrique QualIT.


Cette métrique doit être interprétée comme une métrique basée sur celle NLI, reproductible et légère, permettant une comparaison directe avec QualIT et ROUGE dans un cadre ou on peut comparer les scores des métriques d'un LLM à l'autre.


In [17]:
correlation_nli = df["score_humain"].corr(df["NLI_final"], method="pearson")
print(f"Corrélation (Pearson) entre score humain moyen et métrique NLI_final : {correlation_nli:.2f}")

Corrélation (Pearson) entre score humain moyen et métrique NLI_final : 0.52


La corrélation de Pearson affiche un score de cohérence entre l'évaluation humaine et la métrique NLI de 0.52 : c'est le score le plus faible obtenu jusqu'à maintenant

In [18]:

fig = px.density_contour(
    df, x="NLI_final", y="score_humain",
    nbinsx=20, nbinsy=20
)
fig.update_traces(contours_coloring="fill", contours_showlabels=False, colorscale="Blues")

# Scatter plot
fig.add_scatter(
    x=df["NLI_final"], y=df["score_humain"],
    mode="markers",
    marker=dict(color="#000000"),
    hovertemplate="Métrique NLI_final = %{x}<br>Score humain = %{y}<extra></extra>"
)

# Ligne de référence (x = y) adaptée aux échelles : x∈[0,1], y∈[0,10]
# => y = 10x
fig.add_shape(
    type="line", x0=0, x1=1, y0=0, y1=10,
    xref="x", yref="y",
    line=dict(color="#d41010")
)

fig.update_layout(
    # title="<b>Score humain vs métrique NLI_final</b>",
    xaxis_title="NLI_final metric", yaxis_title="Human score",
    width=700, height=500,
    coloraxis_showscale=False
)

fig.show()


En terme de détection d'hallucination on est vraiment pas mal. Quelques unes ne sont pas détectées mais globalement on les capte bien.